In [1]:
import requests
import pandas as pd
import time
from random import randint, choice

In [2]:
URL_API = "https://graphql-gateway.axieinfinity.com/graphql"

headers = {
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'es-ES,es;q=0.9,en;q=0.8',
    'cache-control': 'no-cache',
    'content-length': '1010',
    'content-type': 'application/json',
    'cookie': '_ga=GA1.2.581236636.1631802371; _gid=GA1.2.1060654813.1631802371; cf_clearance=ps3wfb304R6sKsauq5bsfRJh.xrDvMo2TWlLub0Vf2s-1631892833-0-250; _gat_gtag_UA_150383258_1=1',
    'origin': 'https://marketplace.axieinfinity.com',
    'pragma': 'no-cache',
    'referer': 'https://marketplace.axieinfinity.com/',
    'sec-ch-ua': '"Google Chrome";v="93", " Not;A Brand";v="99", "Chromium";v="93"',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36',
}

In [3]:
query = """query GetAxieBriefList($auctionType: AuctionType, $criteria: AxieSearchCriteria, $from: Int, $sort: SortBy, $size: Int, $owner: String) {
  axies(auctionType: $auctionType, criteria: $criteria, from: $from, sort: $sort, size: $size, owner: $owner) {
    total
    results {
      ...AxieDetail
      transferHistory(from: $from, size: $size) {
        ...TransferRecords
        __typename
      }
      ethereumTransferHistory(from: $from, size: $size) {
        ...TransferRecords
        __typename
      }   
      __typename
    }
    __typename
  }
}

fragment AxieDetail on Axie {
  id
  image
  class
  chain
  name
  birthDate
  bodyShape
  sireId
  sireClass
  matronId
  matronClass
  stage
  title
  breedCount
  level
  stats {
    ...AxieStats
    __typename
  }
  __typename
}

fragment AxieStats on AxieStats {
  hp
  speed
  skill
  morale
  __typename
}

fragment TransferRecords on TransferRecords {
  total
  results {
    from
    to
    timestamp
    txHash
    withPrice
    withPriceUsd
    __typename
  }
  __typename
}
"""

In [4]:
payload = {
    "operationName": "GetAxieBriefList",
    "variables": {
        "from": 0,
        "size": 10,
        "sort": "Latest",
        "auctionType": "Sale",
        "owner": None,
        "criteria": {
            "region": None,
            "parts": None,
            "bodyShapes": None,
            "classes": None,
            "stages": [
                4
            ],
            "numMystic": None,
            "pureness": None,
            "title": None,
            "breedable": None,
            "breedCount": None,
            "hp": [],
            "skill": [],
            "speed": [],
            "morale": []
        }
    },
    "query": query
}

In [5]:
def get_random_ua():
    with open('agents.txt', encoding='utf-8') as f:
        lines = f.readlines()
    return  choice(lines).replace('\n','')

In [6]:
TOTAL = requests.post(URL_API, json=payload).json()['data']['axies']['total']

START = 0
SIZE = 75
NO_REQUESTS = TOTAL // SIZE
CHECKPOINTS = SIZE * 100 # Guarda un Checkpoint cada 5000 requests
EARLYSTOP = 10

col_dtypes = {
    'id':'uint32',
    'sireId':'uint16',
    'matronId':'uint16',
    'birthDate':'uint32',
    'stage':'uint8',
    'breedCount':'uint8',
    'level':'uint8',
    'hp':'uint8',
    'speed':'uint8',
    'skill':'uint8',
    'morale':'uint8',
    'withPrice': 'float32'
}

payload['variables']['size'] = SIZE

df = pd.DataFrame()

earlystop_count = 0
for i in range(0, NO_REQUESTS * SIZE, SIZE):
  
    if i % (10 * SIZE ) == 0: print('--> Getting from', i, 'To', i + SIZE);

    payload['variables']['from'] = i
    headers['user-agent'] = get_random_ua()
    try:
        time.sleep(randint(3,6))
        res = requests.post(URL_API, json=payload, headers=headers)
        if res.status_code == 200:
            axies = res.json()['data']['axies']['results']
        else:
            raise 'Invalid code'

    except:
        print('------> Error Batch #', i, '-', i+SIZE)
        axies = []

    transactions = []
    for axie in axies:
        stats = axie.pop('stats')
        axie = { **axie, **stats}
        txns = axie.pop('ethereumTransferHistory')['results'] + axie.pop('transferHistory')['results']
        transactions += [ {**axie, **t} for t in txns ]

    try:
        transactions = pd.DataFrame(transactions).astype(col_dtypes).drop(columns='__typename')
        transactions.withPrice = transactions.withPrice / 1e18
        df = df.append(transactions)
        earlystop_count = 0
    
    except:
        print('------> Empty Batch #', i, '-', i+SIZE)
        earlystop_count += 1
        if earlystop_count == EARLYSTOP:
            break

    if i and ( i + SIZE ) % CHECKPOINTS == 0 and not df.empty:
        print('---------> Saving checkpoint!')
        df = df.set_index('txHash')
        df.to_csv(f'./dataset/dataset_{ i + SIZE - CHECKPOINTS }_{ i + SIZE }.csv')
        df = pd.DataFrame()

print('---------> Saving last checkpoint!')
if not df.empty: df = df.set_index('txHash');
df.to_csv(f'./dataset/dataset_last.csv')

--> Getting from 0 To 75
------> Error Batch # 300 - 375
------> Empty Batch # 300 - 375
--> Getting from 750 To 825
--> Getting from 1500 To 1575
--> Getting from 2250 To 2325
------> Error Batch # 2325 - 2400
------> Empty Batch # 2325 - 2400
------> Error Batch # 2925 - 3000
------> Empty Batch # 2925 - 3000
--> Getting from 3000 To 3075
------> Error Batch # 3375 - 3450
------> Empty Batch # 3375 - 3450
--> Getting from 3750 To 3825
------> Error Batch # 4425 - 4500
------> Empty Batch # 4425 - 4500
--> Getting from 4500 To 4575
------> Error Batch # 4500 - 4575
------> Empty Batch # 4500 - 4575
------> Error Batch # 4950 - 5025
------> Empty Batch # 4950 - 5025
--> Getting from 5250 To 5325
--> Getting from 6000 To 6075
--> Getting from 6750 To 6825
------> Error Batch # 7275 - 7350
------> Empty Batch # 7275 - 7350
---------> Saving checkpoint!
--> Getting from 7500 To 7575
------> Error Batch # 7950 - 8025
------> Empty Batch # 7950 - 8025
--> Getting from 8250 To 8325
--> Getti

In [8]:
df

,id,image,class,chain,name,birthDate,bodyShape,sireId,sireClass,matronId,...,level,hp,speed,skill,morale,from,to,timestamp,withPrice,withPriceUsd
txHash,,,,,,,,,,,,,,,,,,,,,
0x4ddd7291cff2669622cf7b871efa462702925ade517619b549d4afc174535c99,2836851,https://storage.googleapis.com/assets.axieinfi...,Bird,ronin,[META] HARE/SNIPER 59 SPEED BIRD with CUCKOO,1627312689,Normal,51925,Bird,52835,...,1,27,59,35,43,0xfcfe7ea64967c8f73c0e3ef14f360c92c511bb0c,0x057e353218043544bd9c965931aec208e75d88ce,1632420978,0.070000,219.78775000000002
0x1ffb81cd13379fc2dd576438ae26e284f48b0e2b82438575c3e24cb83cff5738,2836851,https://storage.googleapis.com/assets.axieinfi...,Bird,ronin,[META] HARE/SNIPER 59 SPEED BIRD with CUCKOO,1627312689,Normal,51925,Bird,52835,...,1,27,59,35,43,0x1004f5025f107da93c72cb84d3943fa0dfa24f6c,0x1db97e707312cff3f86ad0965574fa4b10ece471,1628288042,0.177221,556.4424938722511
0xa6e6cc055dded97c2b789a118a072a4d40e135df3eea258a674d6b8079da12ad,2309925,https://storage.googleapis.com/assets.axieinfi...,Plant,ronin,Axie #2309925,1626467058,Normal,57004,Plant,60204,...,1,61,31,31,41,0x5051fdf8b322bac24810f95032b792db60fc20d1,0x1acf4800c7a4e7cf917c7416a6e8d691e9dd92b9,1631152467,0.074000,232.34704999999997
0xe088315e9bd852fa8ac1febc4484134dd1740a6be38917ba173a1698ec0e76a2,2309925,https://storage.googleapis.com/assets.axieinfi...,Plant,ronin,Axie #2309925,1626467058,Normal,57004,Plant,60204,...,1,61,31,31,41,0x357a51d0fd00097611f636c4bf117c20c960ead0,0x5051fdf8b322bac24810f95032b792db60fc20d1,1626990916,0.201802,633.6232263020833
0xe564d6b50904c8113125d8284335cc665de98b4b43b195e90019dae70df45507,4986165,https://storage.googleapis.com/assets.axieinfi...,Reptile,ronin,Razor Buddy,1629921075,Normal,6470,Reptile,6417,...,1,57,41,31,35,0x293882027c938f8e00555af86488e5f5993f09da,0x5234e161bc00d472c1cfbee51019c351cf2b3424,1632390119,0.080000,251.18599999999998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0x02e011d74d22f8bff019f11f71ac04efe1865562c8abbbc90cf900fcb48a18d7,4752099,https://storage.googleapis.com/assets.axieinfi...,Aquatic,ronin,,1629566147,Fuzzy,64669,Aquatic,64421,...,1,44,52,35,33,0x8a300de8048f13ef9c4053ae101804e91c73013b,0x24f90ff121a2d7f7b42ac731c23ecd919a833d65,1630558880,0.011910,37.324302375
0x9885867a179dde8eb0f53bddcbbb408546051237f28b93bd9aa952f52b448139,4752099,https://storage.googleapis.com/assets.axieinfi...,Aquatic,ronin,,1629566147,Fuzzy,64669,Aquatic,64421,...,1,44,52,35,33,0xc56179939eb3e3bde367c895e9c8752ef5116d7c,0x8a300de8048f13ef9c4053ae101804e91c73013b,1630555574,0.113543,355.8283239409722
0xfe46924e3e69c92d82fbd4869686b8875ad9809197d7b3b8859a2fea8e1f696c,5588469,https://storage.googleapis.com/assets.axieinfi...,Reptile,ronin,Axie #5588469,1630871679,Normal,58166,Reptile,29237,...,1,55,40,31,38,0x9f5627ad466710281cc9ca52a2129ddb395ccac8,0x400f7d12ae62b6f00133b1fe0815912bb184d6f5,1631544163,0.384286,1204.2998328819444
